In [45]:
import openai

from ruamel.yaml import YAML
from ruamel.yaml.scalarstring import LiteralScalarString

import json
import os
import time

In [16]:
openai.api_key = os.environ['OPENAI_API_KEY']

yaml=YAML()
yaml.default_flow_style = False

#def str_presenter(dumper, data):
#    """configures yaml for dumping multiline strings
#    Ref: https://stackoverflow.com/questions/8640959/how-can-i-control-what-scalar-form-pyyaml-uses-for-my-data"""
#    if len(data.splitlines()) > 1:  # check for multiline string
#        return dumper.represent_scalar('tag:yaml.org,2002:str', data, style='|')
#    return dumper.represent_scalar('tag:yaml.org,2002:str', data)
#
#yaml.add_representer(str, str_presenter)
#yaml.representer.SafeRepresenter.add_representer(str, str_presenter)

In [50]:
with open('calc-problems.yaml', 'r') as f:
    problems = yaml.load(f)

In [19]:
problems

[{'input': '\\[\n\\int x^n dx\n\\]\nwhere \\(n != -1\\).', 'ideal': '\\(\\int x^n dx = \\frac{x^{n+1}}{n+1} + C\\), for \\(n \\neq -1\\).'}, {'input': '\\[\n\\int \\frac{1}{x} dx\n\\]', 'ideal': '\\(\\int \\frac{1}{x} dx = \\ln|x| + C\\).'}, {'input': '\\[\n\\int e^{x} dx\n\\]', 'ideal': '\\(\\int e^x dx = e^x + C\\).'}, {'input': '\\[\n\\int b^x dx\n\\]', 'ideal': '\\(\\int b^x dx = \\frac{b^x}{\\ln b} + C\\).'}, {'input': '\\[\n\\int \\sin x dx\n\\]', 'ideal': '\\(\\int \\sin x dx = -\\cos x + C\\).'}, {'input': '\\[\n\\int \\cos x dx\n\\]', 'ideal': '\\(\\int \\cos x dx = \\sin x + C\\).'}, {'input': '\\[\n\\int \\sec^2 x dx\n\\]', 'ideal': '\\(\\int \\sec^2 x dx = \\tan x + C\\).'}, {'input': '\\[\n\\int \\csc^2 x dx\n\\]', 'ideal': '\\(\\int \\csc^2 x dx = -\\cot x + C\\).'}, {'input': '\\[\n\\int \\sec x \\tan x dx\n\\]', 'ideal': '\\(\\int \\sec x \\tan x dx = \\sec x + C\\).'}, {'input': '\\[\n\\int \\csc x \\cot x dx\n\\]', 'ideal': '\\(\\int \\csc x \\cot x dx = -\\csc x + C\

In [20]:
problem_fmt = '''
Evaluate the following integral:
{problem}
Explain your reasoning. Use LaTeX with delimiters '\(' and '\)' to make mathematical statements. When you are done, repeat your answer by itself on a new line.
'''

In [52]:
for i, problem in enumerate(problems):
  if i < 2:
    continue
  prompt = problem_fmt.format(problem = problem['input'])
  print(f'*** Prompt: {prompt}')
  completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "user", "content": prompt}
    ],
    n = 5,
  )
  responses = [LiteralScalarString(choice.message.content) for choice in completion.choices]
  problems[i]['completions'] = responses
  for j, response in enumerate(responses):
    print(f'--- Response {j+1}\n')
    print(response)
  with open('calculus-responses.yaml', 'w') as f:
    yaml.dump(problems, f)
  time.sleep(5)

*** Prompt: 
Evaluate the following integral:
\[
\int e^{x} dx
\]
Explain your reasoning. Use LaTeX with delimiters '\(' and '\)' to make mathematical statements. When you are done, repeat your answer by itself on a new line.

--- Response 1

We can evaluate the integral using the integration rule for exponential functions:
\[
\int e^{x} dx = e^{x} + C
\]
where C is the constant of integration. This is because the derivative of $e^{x}$ is itself:
\[
\frac{d}{dx} (e^{x}) = e^{x}
\]
Therefore, the antiderivative of $e^{x}$ is $e^{x}$. 

Hence, 
\[
\int e^{x} dx = e^{x} + C
\]

\[
\int e^{x} dx = e^{x} + C
\]
--- Response 2

We can evaluate the integral using integration by substitution. Let \(u = e^x\), so \(du/dx = e^x\) and \(dx = du/e^x = du/u\). Then we have:
\begin{align*}
\int e^{x} dx &= \int u \frac{du}{u}\\
&= \int du\\
&= u + C\\
&= e^x + C
\end{align*}
Therefore, \(\int e^{x} dx = e^x + C\).

\[\int e^x dx = e^x + C\]
--- Response 3

We can use the fact that the derivative of 

In [59]:
for i, problem in enumerate(problems):
    for j, response in enumerate(problem['completions']):
        problems[i]['completions'][j] = dict(completion = response)


In [60]:
for problem in problems:
    for response in problem['completions']:
        print(response)


{'response': 'Using the power rule of integration, we have:\n\n\\[\n\\int x^n dx = \\frac{x^{n+1}}{n+1} + C\n\\]\n\nwhere \\(C\\) is the constant of integration. \n\nAnswer: \\[\\int x^n dx = \\frac{x^{n+1}}{n+1} + C\\]'}
{'response': 'Using the power rule of integration, we have:\n\\[\n\\int x^n dx = \\frac{x^{n+1}}{n+1} + C\n\\]\nwhere \\(C\\) is the constant of integration. \n\nRepeated answer: \n\\[\n\\int x^n dx = \\frac{x^{n+1}}{n+1} + C\n\\]'}
{'response': 'We use the power rule of integration, which states that for any real number \\(n \\neq -1\\), \\[\\int x^n dx = \\frac{x^{n+1}}{n+1} + C,\\] where \\(C\\) is the constant of integration. \n\nTherefore, \\[\\int x^n dx = \\frac{x^{n+1}}{n+1} + C.\\]\n\n\\[\\int x^n dx = \\frac{x^{n+1}}{n+1} + C\\]'}
{'response': 'We have \n\\begin{align*} \n\\int x^n dx &= \\frac{x^{n+1}}{n+1} +C \\quad (\\text{since } n \\neq -1)\\\\\n&= \\frac{1}{n+1}x^{n+1}+C\n\\end{align*}\n\nTherefore, \n\\[\\int x^n dx = \\frac{1}{n+1}x^{n+1}+C\\]'}
{'re

In [21]:
prompt = problem_fmt.format(problem = problems[0]['input'])
print(prompt)


Evaluate the following integral:
\[
\int x^n dx
\]
where \(n != -1\).
Explain your reasoning. Use LaTeX with delimiters '\(' and '\)' to make mathematical statements. When you are done, repeat your answer by itself on a new line.



In [23]:
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": prompt}
  ],
  n = 5,
)
responses = [choice.message.content for choice in completion.choices]

In [29]:
responses = [choice.message.content for choice in completion.choices]

In [39]:
responses = [LiteralScalarString(choice.message.content) for choice in completion.choices]

In [40]:
for i, response in enumerate(responses):
  print(f'--- Response {i+1}\n')
  print(response)

--- Response 1

Using the power rule of integration, we have:

\[
\int x^n dx = \frac{x^{n+1}}{n+1} + C
\]

where \(C\) is the constant of integration. 

Answer: \[\int x^n dx = \frac{x^{n+1}}{n+1} + C\]
--- Response 2

Using the power rule of integration, we have:
\[
\int x^n dx = \frac{x^{n+1}}{n+1} + C
\]
where \(C\) is the constant of integration. 

Repeated answer: 
\[
\int x^n dx = \frac{x^{n+1}}{n+1} + C
\]
--- Response 3

We use the power rule of integration, which states that for any real number \(n \neq -1\), \[\int x^n dx = \frac{x^{n+1}}{n+1} + C,\] where \(C\) is the constant of integration. 

Therefore, \[\int x^n dx = \frac{x^{n+1}}{n+1} + C.\]

\[\int x^n dx = \frac{x^{n+1}}{n+1} + C\]
--- Response 4

We have 
\begin{align*} 
\int x^n dx &= \frac{x^{n+1}}{n+1} +C \quad (\text{since } n \neq -1)\\
&= \frac{1}{n+1}x^{n+1}+C
\end{align*}

Therefore, 
\[\int x^n dx = \frac{1}{n+1}x^{n+1}+C\]
--- Response 5

If \(n != -1\), then we have 
\[
\int x^n dx = \frac{1}{n+1}x^{n+1}

In [41]:
problems[0]['completions'] = responses
problems[0]

{'input': '\\[\n\\int x^n dx\n\\]\nwhere \\(n != -1\\).', 'ideal': '\\(\\int x^n dx = \\frac{x^{n+1}}{n+1} + C\\), for \\(n \\neq -1\\).', 'responses': ['Using the power rule of integration, we have:\n\n\\[\n\\int x^n dx = \\frac{x^{n+1}}{n+1} + C\n\\]\n\nwhere \\(C\\) is the constant of integration. \n\nAnswer: \\[\\int x^n dx = \\frac{x^{n+1}}{n+1} + C\\]', 'Using the power rule of integration, we have:\n\\[\n\\int x^n dx = \\frac{x^{n+1}}{n+1} + C\n\\]\nwhere \\(C\\) is the constant of integration. \n\nRepeated answer: \n\\[\n\\int x^n dx = \\frac{x^{n+1}}{n+1} + C\n\\]', 'We use the power rule of integration, which states that for any real number \\(n \\neq -1\\), \\[\\int x^n dx = \\frac{x^{n+1}}{n+1} + C,\\] where \\(C\\) is the constant of integration. \n\nTherefore, \\[\\int x^n dx = \\frac{x^{n+1}}{n+1} + C.\\]\n\n\\[\\int x^n dx = \\frac{x^{n+1}}{n+1} + C\\]', 'We have \n\\begin{align*} \n\\int x^n dx &= \\frac{x^{n+1}}{n+1} +C \\quad (\\text{since } n \\neq -1)\\\\\n&= \\fra

In [ ]:
help(yaml)

In [61]:
with open('calculus-responses.yaml', 'w') as f:
  yaml.dump(problems, f)

In [42]:
print(completion.choices[0].message.content)

Using the Power Rule, we have:
\[
\int x^n dx = \frac{x^{n+1}}{n+1} + C
\]
where \(C\) is the constant of integration.

Repeating the answer: 
\[
\int x^n dx = \frac{x^{n+1}}{n+1} + C
\]
